This kernel is alomost same as [this great kernel](https://www.kaggle.com/httpwwwfszyc/bert-in-keras-taming).
In this kernel, I'll show you how to save the fine-tuning model.
maxlen = 50, sample = 1% because of save time.

In [2]:
%load_ext autotime
import numpy as np
import pandas as pd
import os
import sys
import random
import keras
import tensorflow as tf
import json
sys.path.insert(0, '../input/bert-pretrained-models/bert-master/')
import tokenization  #Actually keras_bert contains tokenization part, here just for convenience
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"  # specify which GPU(s) to be used



!cp -r '../input/keras_bert' '../working'
BERT_PRETRAINED_DIR = '../input/bert-pretrained-models/uncased_L-12_H-768_A-12'
print('***** BERT pretrained directory: {} *****'.format(BERT_PRETRAINED_DIR))


***** BERT pretrained directory: ../input/bert-pretrained-models/uncased_L-12_H-768_A-12 *****


## Load raw model

In [4]:
from keras_bert.bert import get_model
from keras_bert.loader import load_trained_model_from_checkpoint
from keras.optimizers import Adam
adam = Adam(lr=2e-5,decay=0.01)
maxlen = 220
print('begin_build')

config_file = os.path.join(BERT_PRETRAINED_DIR, 'bert_config.json')
checkpoint_file = os.path.join(BERT_PRETRAINED_DIR, 'bert_model.ckpt')
model = load_trained_model_from_checkpoint(config_file, checkpoint_file, training=True,seq_len=maxlen)
model.summary(line_length=120)

begin_build
________________________________________________________________________________________________________________________
Layer (type)                           Output Shape               Param #       Connected to                            
Input-Token (InputLayer)               (None, 220)                0                                                     
________________________________________________________________________________________________________________________
Input-Segment (InputLayer)             (None, 220)                0                                                     
________________________________________________________________________________________________________________________
Embedding-Token (TokenEmbedding)       [(None, 220, 768), (30522, 23440896      Input-Token[0][0]                       
________________________________________________________________________________________________________________________
Embedding-Segment (E

## Build classification model

As the Extract layer extracts only the first token where "['CLS']" used to be, we just take the layer and connect to the single neuron output.

## Prepare Data, Training, Predicting

First the model need train data like [token_input,seg_input,masked input], here we set all segment input to 0 and all masked input to 1.

Still I am finding a more efficient way to do token-convert-to-ids

In [5]:
def convert_lines(example, max_seq_length,tokenizer):
    max_seq_length -=2
    all_tokens = []
    all_mask=[]
    longer = 0
    for i in range(example.shape[0]):
      tokens_a = tokenizer.tokenize(example[i])
      if len(tokens_a)>max_seq_length:
        tokens_a = tokens_a[:max_seq_length]
        longer += 1
        mask=[1]*(max_seq_length+2)
      else:
        mask=[1]*(len(tokens_a)+2)+[0]*(max_seq_length-len(tokens_a))
      one_token = tokenizer.convert_tokens_to_ids(["[CLS]"]+tokens_a+["[SEP]"])+[0] * (max_seq_length - len(tokens_a))
      all_tokens.append(one_token)
      all_mask.append(mask)
    print(longer)
    return np.array(all_tokens),np.array(all_mask)
row=None
nb_epochs=5
bsz = 32
dict_path = os.path.join(BERT_PRETRAINED_DIR, 'vocab.txt')
tokenizer = tokenization.FullTokenizer(vocab_file=dict_path, do_lower_case=True)
print('build tokenizer done')
train_df = pd.read_csv('../input/jigsaw-unintended-bias-in-toxicity-classification/train.csv',nrows=row)
train_df['comment_text'] = train_df['comment_text'].replace({r'\s+$': '', r'^\s+': ''}, regex=True).replace(r'\n',  ' ', regex=True)
# train_df = train_df.sample(frac=0.01,random_state = 42)
# train_lines, train_labels = train_df['comment_text'].values, train_df.target.values 

from sklearn.model_selection import train_test_split 
train,train1, train_y, test_y = train_test_split(train_df,train_df["target"].values,test_size = 0.01, random_state = 2019)
train_lines, train_labels = train['comment_text'].values, train.target.values 


identity_columns = [
    'male', 'female', 'homosexual_gay_or_lesbian', 'christian', 'jewish',
    'muslim', 'black', 'white', 'psychiatric_or_mental_illness']
# Overall
weights = np.ones((len(train),)) / 4
# Subgroup
weights += (train[identity_columns].fillna(0).values>=0.5).sum(axis=1).astype(bool).astype(np.int) / 4
# Background Positive, Subgroup Negative
weights += (( (train['target'].values>=0.5).astype(bool).astype(np.int) +
   (train[identity_columns].fillna(0).values<0.5).sum(axis=1).astype(bool).astype(np.int) ) > 1 ).astype(bool).astype(np.int) / 4
# Background Negative, Subgroup Positive
weights += (( (train['target'].values<0.5).astype(bool).astype(np.int) +
   (train[identity_columns].fillna(0).values>=0.5).sum(axis=1).astype(bool).astype(np.int) ) > 1 ).astype(bool).astype(np.int) / 4
loss_weight = 1.0 / weights.mean()

y_train = np.vstack([(train['target'].values>=0.5).astype(np.int),weights]).T
print(y_train)
y_aux_train = train[['target', 'severe_toxicity', 'obscene', 'identity_attack', 'insult', 'threat']].values



print('sample used',train_lines.shape)
token_input,mask_input= convert_lines(train_lines,maxlen,tokenizer)
seg_input = np.zeros((token_input.shape[0],maxlen))
# mask_input = np.ones((token_input.shape[0],maxlen))
print(token_input.shape)
print(seg_input.shape)
print(mask_input.shape)
print('begin training')



from keras.layers import Dense,Input,Flatten,concatenate,Dropout,Lambda
from keras.models import Model
import keras.backend as K
import re
from keras.losses import binary_crossentropy
import codecs

def custom_loss(y_true, y_pred):
    return binary_crossentropy(K.reshape(y_true[:,0],(-1,1)), y_pred) * y_true[:,1]

sequence_output  = model.layers[-1].output
# pool_output = Dense(1, activation='sigmoid',kernel_initializer=keras.initializers.TruncatedNormal(stddev=0.02),name = 'real_output')(sequence_output)

aux_result = Dense(6, activation='sigmoid',kernel_initializer=keras.initializers.TruncatedNormal(stddev=0.02),name = 'aux_output')(sequence_output)

model3  = Model(inputs=model.input, outputs=[pool_output,aux_result])

model3.compile(loss=[custom_loss,'binary_crossentropy'], loss_weights=[loss_weight, 1.0], optimizer=adam)

#model3.compile(loss='binary_crossentropy', optimizer=adam)
model3.summary()




model3.fit([token_input, seg_input, mask_input],[y_train,y_aux_train],batch_size=bsz,epochs=nb_epochs)

# you can save the fine-tuning model by this line.
model3.save_weights('bert_weights.h5')

build tokenizer done
[[0.   0.25]
 [0.   0.25]
 [1.   0.5 ]
 ...
 [0.   0.25]
 [0.   0.25]
 [0.   0.25]]
sample used (1786825,)
40276
(1786825, 220)
(1786825, 220)
(1786825, 220)
begin training
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Input-Token (InputLayer)        (None, 220)          0                                            
__________________________________________________________________________________________________
Input-Segment (InputLayer)      (None, 220)          0                                            
__________________________________________________________________________________________________
Embedding-Token (TokenEmbedding [(None, 220, 768), ( 23440896    Input-Token[0][0]                
__________________________________________________________________________________________________
Embedding-Segm

Epoch 1/5
1786825/1786825 [==============================] - 29549s 17ms/step - loss: 0.3035 - real_output_loss: 0.0592 - aux_output_loss: 0.1135
Epoch 2/5
1786825/1786825 [==============================] - 29560s 17ms/step - loss: 0.2931 - real_output_loss: 0.0570 - aux_output_loss: 0.1103
Epoch 3/5
1786825/1786825 [==============================] - 29530s 17ms/step - loss: 0.2909 - real_output_loss: 0.0565 - aux_output_loss: 0.1096
Epoch 4/5
1786825/1786825 [==============================] - 29539s 17ms/step - loss: 0.2896 - real_output_loss: 0.0562 - aux_output_loss: 0.1093
Epoch 5/5
1786825/1786825 [==============================] - 29466s 16ms/step - loss: 0.2885 - real_output_loss: 0.0559 - aux_output_loss: 0.1090
time: 1d 17h 34min 20s
